Ex 3.4: Blue screen matting. Set up a blue or green background, e.g., by buying a large
piece of colored posterboard. Take a picture of the empty background, and then of the back-
ground with a new object in front of it. Pull the matte using the difference between each
colored pixel and its assumed corresponding background pixel, using one of the techniques
described in Section 3.1.3 or by Smith and Blinn (1996). 

Blue screen (or green screen) matting is a common technique in photography and film to isolate a foreground subject from a uniform background. Once isolated, the foreground can be composited (“placed”) into a new scene.

Below is a practical, step-by-step guide:

### Capture Images:
Background image (IbgIbg​): Photograph the poster board (blue/green) by itself, making sure the lighting is consistent with your foreground shots.
Foreground image (IfgIfg​): Place your object (or person) in front of the same background and capture the photo.

### Read Images into a Program:
For instance, use Python with OpenCV:

import cv2
bg_img = cv2.imread('background.jpg')  # The empty background
fg_img = cv2.imread('foreground.jpg')  # Foreground with background

Ensure both images have the same resolution and align properly.

### Preprocess Images (Optional):

Adjust brightness/contrast, or apply slight blurring to the background image to handle minor noise.
If camera alignment was not perfect, you may need to align or register the images (e.g., using feature matching or manual alignment).

### Compute the Difference and Threshold:

Convert both images into the same color space (e.g., BGR or HSV).
For each pixel xx:
Compute the color difference d(x)d(x).
Compare d(x)d(x) to one or more thresholds (TlowTlow​, ThighThigh​) to assign α(x)α(x).

### Create the Matte (αα-mask):

Store the αα values in a matrix of the same size as your images.
This αα-mask can be a float image ranging from 0 to 1 (for smooth transitions).

### (Optional) Foreground Color Estimation:

If you want a more accurate foreground color F(x)F(x) (especially near edges), you can estimate:
F(x)=Ifg(x)−(1−α(x)) B(x)α(x),
F(x)=α(x)Ifg​(x)−(1−α(x))B(x)​, but only where α(x)≠0α(x)=0.

### Composite the Foreground onto a New Background (Optional Extension):

Let N(x)N(x) be a new background image at pixel xx.
The compositing equation is:
Ifinal(x)=α(x) F(x)  +  (1−α(x)) N(x).
Ifinal​(x)=α(x)F(x)+(1−α(x))N(x).
This step is useful if you want to place your foreground object into a different scene.

In [4]:
import cv2
import numpy as np

# Read the background (empty) and foreground images
bg_img = cv2.imread('background.png')
fg_img = cv2.imread('foreground.png')

# Convert images to float for more precise calculations
bg_float = bg_img.astype(np.float32) / 255.0
fg_float = fg_img.astype(np.float32) / 255.0

# 1. Compute the color difference (Euclidean distance in BGR)
diff = np.sqrt(np.sum((fg_float - bg_float)**2, axis=2))

# 2. Define thresholds (tune these empirically)
T_low = 0.05   # below this -> definitely background
T_high = 0.15  # above this -> definitely foreground

# 3. Initialize alpha matte
alpha = np.zeros_like(diff)

# 4. Hard thresholding for demonstration:
alpha[diff < T_low] = 0.0
alpha[diff > T_high] = 1.0

# 5. Optional: Smooth transition in the [T_low, T_high] range
transition_region = np.where((diff >= T_low) & (diff <= T_high))
alpha[transition_region] = (diff[transition_region] - T_low) / (T_high - T_low)

# alpha is now in [0,1]

# 6. (Optional) Estimate the foreground color F in edge/transition regions
#    We'll do a direct substitution for demonstration, but in real use you want
#    to avoid division by zero. Only do this where alpha > 0.
F = np.zeros_like(fg_float)
non_zero_alpha = alpha > 0.001
F[non_zero_alpha] = (fg_float[non_zero_alpha] - (1 - alpha[non_zero_alpha, None]) * bg_float[non_zero_alpha]) \
                    / alpha[non_zero_alpha, None]

# 7. (Optional) Composite onto a new background
new_bg = cv2.imread('new_background.png').astype(np.float32)/255.0
new_bg = cv2.resize(new_bg, (bg_img.shape[1], bg_img.shape[0]))  # Resize if needed

# The final composited image
I_final = F * alpha[..., None] + new_bg * (1.0 - alpha[..., None])

# Convert alpha and I_final back to 8-bit for visualization/saving
alpha_8u = (alpha * 255).astype(np.uint8)
I_final_8u = np.clip(I_final * 255, 0, 255).astype(np.uint8)

# Save or display the results
cv2.imwrite('alpha.png', alpha_8u)
cv2.imwrite('final_composited.png', I_final_8u)

cv2.imshow('Alpha Matte', alpha_8u)
cv2.imshow('Composited Image', I_final_8u)
cv2.waitKey(0)
cv2.destroyAllWindows()



**Notes on the code:**
- We load **two images**: the background alone and the foreground.  
- We compute a **difference map** (`diff`) to measure how different each pixel is from the known background.  
- We use **two thresholds** to determine whether a pixel is definitely background, definitely foreground, or in a transitional region.  
- We allow for a **smooth alpha ramp** between \(\alpha = 0\) and \(\alpha = 1\) in that transitional region. This helps reduce sharp edges in the matte.  
- Finally, we composite the extracted foreground onto a **new background** (optional step) using the standard compositing formula:
  \[
  I_\text{final} = \alpha \cdot F + (1-\alpha) \cdot N.
  \]

---

### **4. Practical Considerations**

1. **Lighting Conditions:**  
   Ensure that your background is lit **evenly** to avoid large intensity gradients. Any uneven lighting will reduce the effectiveness of simple difference or color keying.

2. **Shadows & Color Spill:**  
   Shadows on the background can lead to increased difference. Similarly, color from the background can spill onto the foreground (particularly with green screens and reflective surfaces). These may require more advanced spill-suppression or shadow-removal techniques.

3. **Image Alignment:**  
   If the camera or background moved slightly between the background and foreground shots, you need to **align** the images or capture them from a fixed camera setup.

4. **Threshold Tuning:**  
   You may need to experiment with threshold values \((T_\text{low}, T_\text{high})\) for best results. Adjusting them can drastically change the quality of the matte.

5. **More Advanced Keying Methods:**  
   Professional keying methods (e.g., **chroma keying**, **Bayesian matting**, **closed-form matting**) use more sophisticated color models and priors to handle transparency, hair, motion blur, etc. The difference keying shown here is a simplified approach but is sufficient for an introductory exercise.

---

## **Summary**

By capturing two images (background-only and foreground-in-place) and comparing their pixel differences, you can derive a **matte** (\(\alpha\)) that identifies foreground vs. background. This technique (sometimes called **difference keying**) is a straightforward application of the matting equation. Once the matte is obtained, you can optionally refine the **foreground colors** and then **composite** your object onto any desired new background.

**Key references:**
- Szeliski, *Computer Vision: Algorithms and Applications*, Section 3.1.3.
- Smith and Blinn (1996), *Blue Screen Matting* in SIGGRAPH.

Feel free to extend this code with more robust methods for shadow detection, spill suppression, and refined color estimation to get cleaner results!
